## 0. Import libraries and dataset

In [280]:
import pandas as pd
import numpy as np
import re

df_app = pd.read_csv("/Users/ivanmera/Documents/Foundations_cs/googleplaystore.csv")
df_app_reviews = pd.read_csv("/Users/ivanmera/Documents/Foundations_cs/googleplaystore_user_reviews.csv")

In [266]:
df_app.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


## 1. Convert the app sizes to a number

In [267]:
df_app[["Size","Size_unit"]] = df_app["Size"].str.extract("([0-9.]*[0-9]+)([a-zA-Z]|\+)", expand=True)
df_app["Size"] = pd.to_numeric(df_app["Size"])
df_app.loc[df_app["Size_unit"] == 'k', "Size"] = df_app["Size"]/1000
df_app = df_app.drop("Size_unit", axis=1)

## 2. Convert the number of installs to a number

In [268]:
df_app["Installs"] = df_app["Installs"].str.replace("+", "")
df_app["Installs"] = df_app["Installs"].str.replace(",", "")
df_app["Installs"] = pd.to_numeric(df_app["Installs"])

## 3. Transform “Varies with device” into a missing value

In [269]:
df_app = df_app.replace({"Varies with device" : np.nan})

## 4. Convert Current Ver and Android Ver into a dotted number

In [270]:
df_app["Current Ver"] = df_app["Current Ver"].str.extract("([0-9.]*[0-9]+)", expand=True)
df_app["Android Ver"] = df_app["Android Ver"].str.extract("([0-9.]*[0-9]+)", expand=True)

## 5. Remove the duplicates

In [271]:
#####--- 5.0 standardization
df_app.insert(1, "App_lowcase", df_app["App"].str.lower())
df_app["App_lowcase"] = df_app["App_lowcase"].str.replace(" ", "")

#####--- 5.1 drop duplicates
df_app = df_app.drop_duplicates(subset= "App_lowcase", keep = "last")      
df_app = df_app.drop("App_lowcase", axis=1)

## 6. For each category, compute the number of apps

In [272]:
app.groupby("Category")["App"].count()

Category
1.9                       1
ART_AND_DESIGN           65
AUTO_AND_VEHICLES        85
BEAUTY                   53
BOOKS_AND_REFERENCE     231
BUSINESS                460
COMICS                   60
COMMUNICATION           387
DATING                  234
EDUCATION               156
ENTERTAINMENT           149
EVENTS                   64
FAMILY                 1972
FINANCE                 366
FOOD_AND_DRINK          127
GAME                   1144
HEALTH_AND_FITNESS      341
HOUSE_AND_HOME           88
LIBRARIES_AND_DEMO       85
LIFESTYLE               382
MAPS_AND_NAVIGATION     137
MEDICAL                 463
NEWS_AND_MAGAZINES      283
PARENTING                60
PERSONALIZATION         392
PHOTOGRAPHY             335
PRODUCTIVITY            424
SHOPPING                260
SOCIAL                  295
SPORTS                  384
TOOLS                   843
TRAVEL_AND_LOCAL        258
VIDEO_PLAYERS           175
WEATHER                  82
Name: App, dtype: int64

## 7. For each category, compute the average rating

In [273]:
app.groupby("Category")["Rating"].mean()

Category
1.9                    19.000000
ART_AND_DESIGN          4.358065
AUTO_AND_VEHICLES       4.190411
BEAUTY                  4.278571
BOOKS_AND_REFERENCE     4.346067
BUSINESS                4.121452
COMICS                  4.155172
COMMUNICATION           4.158537
DATING                  3.970769
EDUCATION               4.389032
ENTERTAINMENT           4.126174
EVENTS                  4.435556
FAMILY                  4.192272
FINANCE                 4.131889
FOOD_AND_DRINK          4.166972
GAME                    4.286326
HEALTH_AND_FITNESS      4.277104
HOUSE_AND_HOME          4.197368
LIBRARIES_AND_DEMO      4.178462
LIFESTYLE               4.094904
MAPS_AND_NAVIGATION     4.051613
MEDICAL                 4.189143
NEWS_AND_MAGAZINES      4.132189
PARENTING               4.300000
PERSONALIZATION         4.335987
PHOTOGRAPHY             4.192114
PRODUCTIVITY            4.211396
SHOPPING                4.259664
SOCIAL                  4.255598
SPORTS                  4.223511
T

## 8. Create two dataframes: one for the genres and one bridging apps and genres. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity

In [274]:
#####--- 8.0 Genres dataframe
df_genres = pd.DataFrame({"Genres" : df_app["Genres"]})
df_genres["Genres"] = df_genres['Genres'].str.split(';')
df_genres  = df_genres.explode("Genres")

#####--- 8.1 Bridging dataframe
df_app_genres = pd.DataFrame({"App": df_app["App"],
                            "Genres" : df_app["Genres"]})
df_app_genres["Genres"] = df_app_genres['Genres'].str.split(';')
df_app_genres  = df_app_genres.explode("Genres")

## 9. For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)


In [275]:
binary = pd.get_dummies(df_app["Genres"]).astype(bool)
df_app_bool = df_app.join(binary)

## 10. For each genre, compute the average rating. What is the genre with highest average?

In [276]:
df_app.groupby("Genres")["Rating"].mean().sort_values(ascending = False)

Genres
Board;Pretend Play                 4.8
Comics;Creativity                  4.8
Health & Fitness;Education         4.7
Adventure;Brain Games              4.6
Puzzle;Education                   4.6
                                  ... 
Parenting;Brain Games              3.8
Art & Design;Action & Adventure    NaN
Books & Reference;Creativity       NaN
Role Playing;Education             NaN
Trivia;Education                   NaN
Name: Rating, Length: 118, dtype: float64

In [277]:
df_app.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0,Teen,Art & Design,"June 8, 2018",NaN,4.2
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6,50000,Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3


## 11. For each app, compute the approximate income, obtain as a product of number of installs and price.

In [36]:
df_app["Price"] = df_app["Price"].str.replace("$", "")
df_app["Price"] = pd.to_numeric(df_app["Price"])
df_app["Income"] = df_app["Price"] * df_app["Installs"]

## 12. For each app, compute its minimum and maximum Sentiment_polarity

In [281]:
df_app_reviews.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [282]:
df_app_reviews.groupby("App")["Sentiment_Polarity"].max().head()

App
10 Best Foods for You               1.000000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室    0.910000
11st                                1.000000
1800 Contacts - Lens Store          0.838542
1LINE – One Line with One Touch     1.000000
Name: Sentiment_Polarity, dtype: float64

In [283]:
df_app_reviews.groupby("App")["Sentiment_Polarity"].min().head()

App
10 Best Foods for You              -0.8000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室   -0.1125
11st                               -1.0000
1800 Contacts - Lens Store         -0.3000
1LINE – One Line with One Touch    -0.8250
Name: Sentiment_Polarity, dtype: float64